In [1]:
import pandas as pd
import numpy as np

import requests

from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
import time
import os
import re

In [2]:
chromedriver = "/home/michael/chromedriver"
os.environ["webdriver.chrome.driver"] = chromedriver

chrome_options = Options()  
chrome_options.add_argument("--headless")  
chrome_options.add_argument('--dns-prefetch-disable')

# 1.0 Scraping Data
I need to scrape the dates and times for different sporting events in chicago from their respective sites

## 1.1 Bulls

In [6]:
def bulls_seasondata(startyear, years = 2):
    '''Function to pull all games for given year range for the bulls from baskeball reference'''
    columns = ['games','date','time','blank','blank2','location','opponent','result','timing','pts','oppts','w','l','streak','notes']
    finaldata = pd.DataFrame(columns = columns)
    driver = webdriver.Chrome(chromedriver,chrome_options=chrome_options)
    
    for year in range(years):
        driver.get('https://www.basketball-reference.com/teams/CHI/{}_games.html'.format(startyear+year))
        time.sleep(3)
        seasondata = driver.find_element_by_xpath('//table[@id="games"]').get_attribute('outerHTML')
        newdata = pd.read_html(seasondata, header = 0)[0]
        newdata.columns = columns
        finaldata = finaldata.append(newdata)
        try:
            playoffdata = driver.find_element_by_xpath('//table[@id="games_playoffs"]').get_attribute('outerHTML')
            newdata = pd.read_html(playoffdata, header = 0)[0]
            newdata.columns = columns
            finaldata = finaldata.append(newdata)
        except:
            pass
    driver.close()
    return finaldata

In [7]:
bullsdata = bulls_seasondata(2014, years = 5)

In [8]:
bullsdata.head()

,games,date,time,blank,blank2,location,opponent,result,timing,pts,oppts,w,l,streak,notes
0,1,"Tue, Oct 29, 2013",8:00p ET,NaN,Box Score,@,Miami Heat,L,NaN,95,107,0,1,L 1,NaN
1,2,"Thu, Oct 31, 2013",8:00p ET,NaN,Box Score,NaN,New York Knicks,W,NaN,82,81,1,1,W 1,NaN
2,3,"Sat, Nov 2, 2013",7:30p ET,NaN,Box Score,@,Philadelphia 76ers,L,NaN,104,107,1,2,L 1,NaN
3,4,"Wed, Nov 6, 2013",7:00p ET,NaN,Box Score,@,Indiana Pacers,L,NaN,80,97,1,3,L 2,NaN
4,5,"Fri, Nov 8, 2013",8:00p ET,NaN,Box Score,NaN,Utah Jazz,W,NaN,97,73,2,3,W 1,NaN


In [51]:
bullsdata.to_pickle('/home/michael/Documents/Projects/divvydataproject/data/SportballGames/bulls.pkl')

## 1.2 Cubs/Sox

In [21]:
def baseball_seasondata(startyear, years = 2):
    '''Function to pull all games for given year range for the bulls from baskeball reference'''
    basecolumns = ['games','date','blank2','team','location','opponent','result','runsfor','runsagainst','innings',
                   'record','rank','gb','win','loss','save','gametime','day/night','attend','streak','origsch','year']
    finaldata = pd.DataFrame(columns = basecolumns)
    driver = webdriver.Chrome(chromedriver,chrome_options=chrome_options)
    teams = ['CHC','CHW']
    
    for team in teams:
        for year in range(years):
            driver.get('https://www.baseball-reference.com/teams/{}/{}-schedule-scores.shtml'.format(team, startyear+year))
            time.sleep(3)
            seasondata = driver.find_element_by_xpath('//table[@id="team_schedule"]').get_attribute('outerHTML')
            newdata = pd.read_html(seasondata, header = 0)[0]
            newdata['year'] = startyear + year
            newdata.columns = basecolumns
            finaldata = finaldata.append(newdata)
    driver.close()
    return finaldata

In [19]:
newdata.head()

,Gm#,Date,Unnamed: 2,Tm,Unnamed: 4,Opp,W/L,R,RA,Inn,...,Rank,GB,Win,Loss,Save,Time,D/N,Attendance,Streak,Orig. Scheduled
0,1,"Monday, Apr 1",boxscore,CHC,@,PIT,W,3,1,NaN,...,1,Tied,Samardzija,Burnett,Fujikawa,2:59,D,39078,+,NaN
1,2,"Wednesday, Apr 3",boxscore,CHC,@,PIT,L,0,3,NaN,...,1,Tied,Rodriguez,Jackson,Grilli,2:55,N,27667,-,NaN
2,3,"Thursday, Apr 4",boxscore,CHC,@,PIT,W,3,2,NaN,...,1,Tied,Wood,McDonald,Marmol,2:41,D,11634,+,NaN
3,4,"Friday, Apr 5",boxscore,CHC,@,ATL,L,1,4,NaN,...,2,1.0,Minor,Feldman,Kimbrel,2:36,N,33443,-,NaN
4,5,"Saturday, Apr 6",boxscore,CHC,@,ATL,L-wo,5,6,NaN,...,2,1.0,O'Flaherty,Marmol,NaN,3:12,N,38498,--,NaN


In [22]:
baseballdata = baseball_seasondata(2013, years = 5)

In [24]:
baseballdata.tail()

,games,date,blank2,team,location,opponent,result,runsfor,runsagainst,innings,...,gb,win,loss,save,gametime,day/night,attend,streak,origsch,year
164,160,"Friday, Sep 29",boxscore,CHW,@,CLE,L,1,10,NaN,...,35.0,Bauer,Pelfrey,NaN,2:57,N,26983,-,NaN,2017
165,161,"Saturday, Sep 30",boxscore,CHW,@,CLE,W,2,1,NaN,...,34.0,Fulmer,Clevinger,Minaya,3:13,N,33173,+,NaN,2017
166,Gm#,October,NaN,Tm,NaN,Opp,W/L,R,RA,Inn,...,GB,Win,Loss,Save,Time,D/N,Attendance,Streak,Orig. Scheduled,2017
167,162,"Sunday, Oct 1",boxscore,CHW,@,CLE,L,1,3,NaN,...,35.0,Tomlin,Volstad,Allen,2:37,D,30036,-,NaN,2017
168,Gm#,Date,NaN,Tm,NaN,Opp,W/L,R,RA,Inn,...,GB,Win,Loss,Save,Time,D/N,Attendance,Streak,Orig. Scheduled,2017


In [25]:
baseballdata.to_pickle('/home/michael/Documents/Projects/divvydataproject/data/SportballGames/baseball.pkl')

## 1.3 Bears

In [3]:
def bears_seasondata(startyear, years = 2):
    '''Function to pull all games for given year range for the bulls from baskeball reference'''
    bearcol = ['games','day','date','time','blank2','result','ot','record','location','opponent','pts', 'oppts',
               '1stD', 'TotYd', 'PassY', 'RushY', 'TO', '1stD.1', 'TotYd.1', 'PassY.1', 'RushY.1', 'TO.1', 'Offense', 'Defense', 'Sp. Tms', 'year']
    finaldata = pd.DataFrame(columns = bearcol)
    driver = webdriver.Chrome(chromedriver,chrome_options=chrome_options)
        
    for year in range(years):
        driver.get('https://www.pro-football-reference.com/teams/chi/{}_games.htm'.format(startyear+year))
        time.sleep(3)
        seasondata = driver.find_element_by_xpath('//table[@id="games"]').get_attribute('outerHTML')
        newdata = pd.read_html(seasondata, header = 0)[0]
        newdata['year'] = startyear + year
        newdata.columns = bearcol
        finaldata = finaldata.append(newdata)
    driver.close()
    return finaldata

In [4]:
bearsdata = bears_seasondata(2013, years = 5)

In [5]:
bearsdata.head()

,games,day,date,time,blank2,result,ot,record,location,opponent,...,TO,1stD.1,TotYd.1,PassY.1,RushY.1,TO.1,Offense,Defense,Sp. Tms,year
0,Week,Day,Date,NaN,NaN,NaN,OT,Rec,NaN,Opp,...,TO,1stD,TotYd,PassY,RushY,TO,Offense,Defense,Sp. Tms,2013
1,1,Sun,September 8,1:02PM ET,boxscore,W,NaN,1-0,NaN,Cincinnati Bengals,...,1,18,340,277,63,3,-0.93,-2.65,3.28,2013
2,2,Sun,September 15,1:03PM ET,boxscore,W,NaN,2-0,NaN,Minnesota Vikings,...,4,19,350,227,123,3,-0.92,3.26,1.56,2013
3,3,Sun,September 22,8:30PM ET,boxscore,W,NaN,3-0,@,Pittsburgh Steelers,...,NaN,21,459,379,80,5,1.41,15.78,0.13,2013
4,4,Sun,September 29,1:03PM ET,boxscore,L,NaN,3-1,@,Detroit Lions,...,4,23,387,228,159,3,-2.72,1.64,-6.73,2013


In [6]:
bearsdata.shape

(90, 26)

In [7]:
bearsdata.to_pickle('/home/michael/Documents/Projects/divvydataproject/data/SportballGames/bears.pkl')

## 1.4 Blackhawks

In [12]:
def hawks_seasondata(startyear, years = 2):
    '''Function to pull all games for given year range for the bulls from baskeball reference'''
    columns = ['games','date','time','blank','blank2','location','opponent','result','timing','pts','oppts','w','l','streak','notes']
    finaldata = pd.DataFrame(columns = columns)
    driver = webdriver.Chrome(chromedriver,chrome_options=chrome_options)
    
    for year in range(years):
        driver.get('https://www.hockey-reference.com/teams/CHI/{}_games.html'.format(startyear+year))
        time.sleep(3)
        seasondata = driver.find_element_by_xpath('//table[@id="games"]').get_attribute('outerHTML')
        newdata = pd.read_html(seasondata, header = 1)[0]
        newdata.columns = columns
        finaldata = finaldata.append(newdata)
        try:
            playoffdata = driver.find_element_by_xpath('//table[@id="games_playoffs"]').get_attribute('outerHTML')
            newdata = pd.read_html(playoffdata, header = 0)[0]
            newdata.columns = columns
            finaldata = finaldata.append(newdata)
        except:
            pass
    driver.close()
    return finaldata

In [13]:
driver = webdriver.Chrome(chromedriver,chrome_options=chrome_options)
driver.get('https://www.hockey-reference.com/teams/CHI/2014_games.html')
seasondata = driver.find_element_by_xpath('//table[@id="games"]').get_attribute('outerHTML')
newdata = pd.read_html(seasondata, header = 1)[0]

In [14]:
newdata.head()

,GP,Date,Time,Unnamed: 3,Opponent,GF,GA,Unnamed: 7,Unnamed: 8,W,...,S,PIM,PPG,SHG,Unnamed: 18,S.1,PIM.1,PPG.1,SHG.1,Notes
0,1,2013-10-01,8:00 PM,NaN,Washington Capitals,6,4,W,NaN,1,...,34,12,1,0,NaN,32,8,3,0,NaN
1,2,2013-10-05,8:00 PM,NaN,Tampa Bay Lightning,2,3,L,SO,1,...,39,2,1,0,NaN,16,10,1,0,NaN
2,3,2013-10-09,8:00 PM,@,St. Louis Blues,2,3,L,NaN,1,...,28,13,2,0,NaN,34,13,1,0,NaN
3,4,2013-10-11,8:00 PM,NaN,New York Islanders,3,2,W,NaN,2,...,28,6,0,0,NaN,19,10,0,0,NaN
4,5,2013-10-12,8:00 PM,NaN,Buffalo Sabres,2,1,W,NaN,3,...,40,8,1,0,NaN,29,12,0,0,NaN


In [15]:
newdata.columns

Index(['GP', 'Date', 'Time', 'Unnamed: 3', 'Opponent', 'GF', 'GA',
       'Unnamed: 7', 'Unnamed: 8', 'W', 'L', 'OL', 'Streak', 'Unnamed: 13',
       'S', 'PIM', 'PPG', 'SHG', 'Unnamed: 18', 'S.1', 'PIM.1', 'PPG.1',
       'SHG.1', 'Notes'],
      dtype='object')

In [ ]:
gamecolumns = ['games','date','time','locations','opponent','pts','oppts','result','shootout','wins','losses','w','l','streak','notes']